# Marketing Spend Efficiency
Evaluating the cost per lead and cost per acquisition for a for-profit college across their current marketing mix. For this client, an acquisition is an enrollment.

In [73]:
import pandas as pd
from pandas.tseries.offsets import DateOffset, MonthEnd

# override scientific notation
pd.options.display.float_format = '{:.2f}'.format

In [6]:
# import and clean marketing spend data
spend = pd.read_csv('marketing_spend.csv', names = ['date', 'channel', 'spend_usd'], header=0)

spend['spend_usd'] = spend['spend_usd'].str.replace('$', '').str.replace('-', '').str.replace(',', '').str.strip().replace('', '0').astype(float)
spend['date'] = pd.to_datetime(spend['date'])
spend['channel'] = spend['channel'].str.lower()

In [78]:
# import and clean applicant data
cols = ['person_id', 'utm_marketing_channel', 'reported_channel', 'lead_created_date', 'app_submit_date', 'admit_date', 'enroll_date', 'registration_status', 'last_date_attendance', 'quarter_start_date', 'quarter_end_date']
applicants = pd.read_csv('applicants_abbrev.csv', names=cols, header=0)

date_cols = ['lead_created_date', 'app_submit_date', 'admit_date', 'enroll_date', 'last_date_attendance', 'quarter_start_date', 'quarter_end_date']
for col in date_cols:
    applicants[col] = pd.to_datetime(applicants[col])

In [79]:
applicants.sample(5)

,person_id,utm_marketing_channel,reported_channel,lead_created_date,app_submit_date,admit_date,enroll_date,registration_status,last_date_attendance,quarter_start_date,quarter_end_date
11734,11791,facebook,facebook,2024-08-21,2024-08-24,NaT,NaT,NaN,NaT,2024-10-07,2024-12-12
8252,8300,facebook,email,2024-08-28,2024-08-28,NaT,NaT,NaN,NaT,2024-10-07,2024-12-12
6520,6565,facebook,facebook,2024-09-07,2024-09-07,2024-09-16,2024-09-18,Registered,2024-12-11,2024-10-07,2024-12-12
2567,2581,facebook,facebook,2024-09-06,2024-09-06,NaT,NaT,NaN,NaT,2024-10-07,2024-12-12
8975,9024,facebook,facebook,2024-09-08,2024-09-08,NaT,NaT,NaN,NaT,2024-10-07,2024-12-12


## What is the cost per enrollment across different marketing channels?

In [57]:
# aggregate spend by channel
by_channel_spend = spend.groupby('channel')['spend_usd'].sum().to_frame()

In [58]:
# aggregate applications and enrollments by utm channel
by_utm_apps = applicants.groupby('utm_marketing_channel').agg(
    applications  = ('app_submit_date', 'count')
    , enrollments = ('enroll_date', 'count')
)

# join spend and conversion data
by_utm = pd.merge(by_utm_apps, by_channel_spend, left_index=True, right_index=True, how='outer')
by_utm['cpe'] = by_utm['spend_usd'] / by_utm['enrollments']

In [59]:
# aggregate applications and enrollments by self-reported channel
by_reported_apps = applicants.groupby('reported_channel').agg(
    applications  = ('app_submit_date', 'count')
    , enrollments = ('enroll_date', 'count')
)

# join spend and conversion data
by_reported = pd.merge(by_reported_apps, by_channel_spend, left_index=True, right_index=True, how='outer')
by_reported['cpe'] = by_reported['spend_usd'] / by_reported['enrollments']

In [62]:
by_utm.sort_values('enrollments', ascending=False)

,applications,enrollments,spend_usd,cpe
facebook,6967.00,415.00,5148271.00,12405.47
unknown,2224.00,228.00,NaN,NaN
google,1070.00,89.00,1064115.54,11956.35
snapchat,810.00,56.00,914669.70,16333.39
employer,466.00,54.00,NaN,NaN
tiktok,1098.00,39.00,1916758.90,49147.66
natural search,267.00,29.00,NaN,NaN
referral,2.00,1.00,NaN,NaN
linkedin,2.00,0.00,7494.60,inf
other,NaN,NaN,0.00,NaN


### Cost per Enrollment for Paid Channels, from 6/15/24 to 9/15/24, according to UTM Channel

In [65]:
# transpose and drop nulls for clarity
by_utm[by_utm['cpe'].notnull()].transpose()

,facebook,google,linkedin,snapchat,tiktok
applications,6967.00,1070.00,2.00,810.00,1098.00
enrollments,415.00,89.00,0.00,56.00,39.00
spend_usd,5148271.00,1064115.54,7494.60,914669.70,1916758.90
cpe,12405.47,11956.35,inf,16333.39,49147.66


### Cost per Enrollment for Paid Channels, from 6/15/24 to 9/15/24, according to 'How did you hear about us?' responses

In [66]:
by_reported[by_reported['cpe'].notnull()].transpose()

,facebook,google,linkedin,other,snapchat,tiktok
applications,7470.00,877.00,69.00,164.00,647.00,1739.00
enrollments,496.00,73.00,3.00,16.00,44.00,92.00
spend_usd,5148271.00,1064115.54,7494.60,0.00,914669.70,1916758.90
cpe,10379.58,14576.93,2498.20,0.00,20787.95,20834.34


## Cost per Enrollment by Month

#### Calculations

In [77]:
applicants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12920 entries, 0 to 12919
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   person_id              12920 non-null  int64 
 1   utm_marketing_channel  12920 non-null  object
 2   reported_channel       12920 non-null  object
 3   lead_created_date      12916 non-null  object
 4   app_submit_date        12906 non-null  object
 5   admit_date             1882 non-null   object
 6   enroll_date            911 non-null    object
 7   registration_status    911 non-null    object
 8   last_date_attendance   755 non-null    object
 9   quarter_start_date     12920 non-null  object
 10  quarter_end_date       12920 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.2+ MB


In [71]:
# helper function for grouping leads and conversions by month ending 15th
def get_next_15th(date):
    if date.day <= 15:
        return pd.Timestamp(date.year, date.month, 15)
    else:
        return pd.Timestamp(date.year, date.month, 1) + MonthEnd(1) + pd.DateOffset(days=15)

In [80]:
spend['month_end_15'] = spend['date'].apply(get_next_15th)
applicants['month_end_15'] = applicants['lead_created_date'].apply(get_next_15th)

# re-assign 6/15 to month ending 7/15 to avoid having a month with only one day
# added benefit: makes all 3 months of this period 31 days long
spend.loc[spend['date'] == '2024-06-15', 'month_end_15'] = pd.to_datetime('2024-07-15')
applicants.loc[applicants['lead_created_date'] == '2024-06-15', 'month_end_15'] = pd.to_datetime('2024-07-15')

In [81]:
# aggregate spend per (channel, month)
by_channel_spend_by_month = spend.groupby(['channel', 'month_end_15'])['spend_usd'].sum().to_frame().reset_index()

In [82]:
# aggregate number of applications and enrollments per (utm_marketing_channel, month)
by_utm_apps_by_month = applicants.groupby(['utm_marketing_channel', 'month_end_15']).agg(
    applications  = ('app_submit_date', 'count')
    , enrollments = ('enroll_date', 'count')
).reset_index()

In [83]:
# join spend and number of apps/enrolls per (channel, month)
by_utm_by_month = pd.merge(
    by_utm_apps_by_month
    , by_channel_spend_by_month
    , left_on=['utm_marketing_channel', 'month_end_15']
    , right_on=['channel', 'month_end_15']
    , how='outer')
by_utm_by_month['cpe'] = by_utm_by_month['spend_usd'] / by_utm_by_month['enrollments']
by_utm_by_month['cpl'] = by_utm_by_month['spend_usd'] / by_utm_by_month['applications']

### Spend, COL, CPE per UTM Channel per Month ending 15th

In [86]:
by_utm_by_month[by_utm_by_month['spend_usd'].notnull()] \
    .loc[:, ['utm_marketing_channel', 'month_end_15', 'spend_usd', 'cpe', 'cpl']]

,utm_marketing_channel,month_end_15,spend_usd,cpe,cpl
19,facebook,2024-07-15,701986.30,14935.88,926.10
20,facebook,2024-08-15,2200297.70,13019.51,766.12
21,facebook,2024-09-15,2245987.00,12075.20,718.72
32,google,2024-07-15,244865.95,10202.75,903.56
33,google,2024-08-15,507336.00,13008.62,1110.14
34,google,2024-09-15,311913.59,12996.40,1002.94
37,NaN,2024-07-15,7494.60,NaN,NaN
38,NaN,2024-08-15,0.00,NaN,NaN
39,NaN,2024-09-15,0.00,NaN,NaN
46,NaN,2024-07-15,0.00,NaN,NaN
